# Gitops example

# Install deployment controller with gitops enabled

In [ ]:
! helm install mlflow-controller charts/mlflow-controller  -n mlflow --set gitops.enabled=true   --set mlflow.backend=blob

# Register Mlflow models

In [ ]:
import os

import mlflow
import mlflow.sklearn
import pandas as pd
from minio import Minio
from mlflow.tracking import MlflowClient
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split


os.environ['MLFLOW_TRACKING_URI']='http://localhost:5000'
os.environ['AZURE_STORAGE_ACCESS_KEY']=""
os.environ['AZURE_STORAGE_CONNECTION_STRING']= ""

def main(MODEL_NAME="iris gitops", stage="Staging"):

    iris = datasets.load_iris()
    iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)
    y = iris.target
    iris_df["target"] = y

    print(iris_df.head())

    train_df, test_df = train_test_split(
        iris_df, test_size=0.3, random_state=42, stratify=iris_df["target"]
    )
    X_train = train_df[
        [
            "sepal length (cm)",
            "sepal width (cm)",
            "petal length (cm)",
            "petal width (cm)",
        ]
    ]
    y_train = train_df["target"]

    X_test = test_df[
        [
            "sepal length (cm)",
            "sepal width (cm)",
            "petal length (cm)",
            "petal width (cm)",
        ]
    ]
    y_test = test_df["target"]

    EXPERIMENT_NAME = MODEL_NAME

    print("IRIS train df shape")
    print(X_train.shape)
    print(y_train.shape)

    print("IRIS test df shape")
    print(X_test.shape)
    print(y_test.shape)

    mlflow_client = MlflowClient()

    # Create an MLFlow experiment, if not already exists
    experiment_details = mlflow_client.get_experiment_by_name(EXPERIMENT_NAME)

    if experiment_details is not None:
        experiment_id = experiment_details.experiment_id
    else:
        experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)

    # Start an MLFlow experiment run
    with mlflow.start_run(
        experiment_id=experiment_id, run_name="iris dataset rf run"
    ) as run:
        # Log parameters

        mlflow.log_param("max_depth", 10)
        mlflow.log_param("random_state", 0)
        mlflow.log_param("n_estimators", 100)
        clf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=0)
        clf.fit(X_train, y_train)
        iris_predict_y = clf.predict(X_test)

        roc_auc_score_val = roc_auc_score(
            y_test, clf.predict_proba(X_test), multi_class="ovr"
        )
        mlflow.log_metric("test roc_auc_score", roc_auc_score_val)

        # Log model
        result = mlflow.sklearn.log_model(clf, artifact_path="model")

        # Register a new version
    result = mlflow.register_model(result.model_uri, MODEL_NAME)

    client = MlflowClient()
    client.transition_model_version_stage(
        name=MODEL_NAME, version=result.version, stage=stage
    )



for i in range(5):
    main(MODEL_NAME=f"iris demo{i}")


# write deployment file and commit to git repository

In [ ]:
! git clone https://github.com/rocket9-code/model-deployments

In [ ]:
dep_yaml ="""apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: mlflow-var-test1
spec:
  name: iris
  predictors:
  - graph:
      children:
        - name: step-one
          modelUri: '{{ mlflow.blob["iris demo1"] }}'
          envSecretRefName: seldon-rclone-secret
          implementation: MLFLOW_SERVER
          type: MODEL
          children: 
              - name: step-two
                modelUri: '{{ mlflow.blob["iris demo2"] }}'
                envSecretRefName: seldon-rclone-secret
                implementation: MLFLOW_SERVER
                type: MODEL
                children: []
        - name: step-three
          implementation: MLFLOW_SERVER
          modelUri: '{{ mlflow.blob["iris demo3"] }}'
          envSecretRefName: seldon-rclone-secret
          type: MODEL
          children: []
      implementation: MLFLOW_SERVER
      modelUri: '{{ mlflow.blob["iris demo4"] }}'
      envSecretRefName: seldon-rclone-secret
      logger:
        url: http://broker-ingress.knative-eventing.svc.cluster.local/demo/default
        mode: all
      name: classifier
    name: default
    replicas: 1"""
with open('seldon-deploy-test1.yaml', 'x') as f:
    f.write(dep_yaml)

In [ ]:
! git add seldon-deploy-test1.yaml

In [ ]:
! git commit -m "test deploy yaml" 
! git push

# wait for the controller to pickup the changes and creates a new deploy yaml

In [ ]:
import time

from kubernetes import client as KubeClient
from kubernetes import config
try:
    config.load_kube_config()
except config.ConfigException:
    config.load_incluster_config()
kube_client = KubeClient.CustomObjectsApi()

In [ ]:
manifest = kube_client.get_namespaced_custom_object(
    group="machinelearning.seldon.io",
    version="v1",
    plural="seldondeployments",
    namespace="staging",
    name="mlflow-var-test1",
)
demo1 = manifest["spec"]["predictors"][0]["graph"]["children"][0]["modelUri"]
demo2 = manifest["spec"]["predictors"][0]["graph"]["children"][0]["children"][0][
    "modelUri"
]
demo3 = manifest["spec"]["predictors"][0]["graph"]["children"][1]["modelUri"]
demo4 = manifest["spec"]["predictors"][0]["graph"]["modelUri"]

print(demo1, demo2, demo3, demo4)
